# The purpose of this notebook is to create datasets consisting of bitcoin prices together with lagged tweets

In [1]:
#imports
import pandas as pd 
from datetime import datetime
from datetime import timedelta

### Load BTC data with dates

In [2]:
btc_data = pd.read_csv("datasets/general/BTCDATAwithdate.csv") 

In [3]:
btc_data.head()

,Unnamed: 0,Close,Volume_(BTC),Volume_(Currency),Weighted_Price,timestamp
0,3500000,6936.26,1.001363,6944.466619,6935.011702,2018-08-31 11:04:00+00:00
1,3500001,6944.90,10.379585,72022.107341,6938.823172,2018-08-31 11:05:00+00:00
2,3500002,6949.89,1.264825,8789.841563,6949.454444,2018-08-31 11:06:00+00:00
3,3500003,6950.61,1.948446,13550.262119,6954.393879,2018-08-31 11:07:00+00:00
4,3500004,6949.51,5.503697,38235.502385,6947.240108,2018-08-31 11:08:00+00:00
...,...,...,...,...,...,...
1072252,4572252,10325.89,0.081143,837.754188,10324.419779,2020-09-14 01:56:00+00:00
1072253,4572253,10331.97,0.572817,5918.028741,10331.444396,2020-09-14 01:57:00+00:00
1072254,4572254,10331.47,2.489909,25711.238323,10326.175283,2020-09-14 01:58:00+00:00
1072255,4572255,10334.78,4.572660,47253.747619,10333.972651,2020-09-14 01:59:00+00:00


In [16]:
#Change timestamp to data in bitcoin data
# btc_data['timestamp'] = [datetime.fromtimestamp(x for x in btc_data['Timestamp']]
btc_data['timestamp'] = pd.to_datetime(btc_data['timestamp'], utc = True)  

## According to the analyser used to create polarity scores, load the file

In [4]:
#this can either be TextBlob or vader
analyser = "TextBlob"
# polarity_medium = "vader"

#get tweets
tweet_data = pd.read_csv("./../datasets/general/"+analyser+"/tweets.csv")

In [6]:
tweet_data.head()

,Unnamed: 0,id,user,fullname,url,timestamp,replies,likes,retweets,text,polarity,subjectivity
0,0,1132977055300300800,KamdemAbdiel,Abdiel kamdem,NaN,2019-05-27 11:49:14+00,0.0,0.0,0.0,È appena uscito un nuovo video! LES CRYPTOMONN...,0.000000,0.000000
1,1,1132977073402736640,bitcointe,Bitcointe,NaN,2019-05-27 11:49:18+00,0.0,0.0,0.0,Cardano: Digitize Currencies; EOS https://t.co...,0.000000,0.000000
2,2,1132977023893139456,3eyedbran,Bran - 3 Eyed Raven,NaN,2019-05-27 11:49:06+00,0.0,2.0,1.0,Another Test tweet that wasn't caught in the s...,0.000000,0.000000
3,3,1132977089089556481,DetroitCrypto,J. Scardina,NaN,2019-05-27 11:49:22+00,0.0,0.0,0.0,Current Crypto Prices! \n\nBTC: $8721.99 USD\n...,0.000000,0.400000
4,4,1132977092340191232,mmursaleen72,Muhammad Mursaleen,NaN,2019-05-27 11:49:23+00,0.0,0.0,0.0,Spiv (Nosar Baz): BITCOIN Is An Asset &amp; NO...,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
20410261,20165008,1.19826e+18,JacobCanfield,Jacob Canfield,NaN,2019-11-23 15:28:50+00,2.0,16.0,3.0,Happy #FibonacciDay \n\nA while back I created...,0.188226,0.517929
20410262,20165009,1.19827e+18,Vizique,Vizique,NaN,2019-11-23 15:45:55+00,0.0,0.0,0.0,Bitcoin Suisse Certificates :) https://t.co/nd...,0.500000,1.000000
20410263,20165010,1.19827e+18,torusJKL,Gal Buki ($torusJKL),NaN,2019-11-23 15:45:56+00,0.0,0.0,0.0,Register now for the early access of the Codug...,0.100000,0.300000
20410264,20165011,1.19827e+18,Adekunl95628158,Adekunle Daniel,NaN,2019-11-23 15:45:57+00,0.0,0.0,0.0,@btc \n@btc \nDo you know that BTC Baskets isn...,0.240625,0.500000


### Create lag

In [ ]:
#choose lag
lag = 1
#thsi can be hours or days
granularity = "hours"

In [20]:
#copy tweet data and move by lag
tweet_data_copy = tweet_data.copy()
tweet_data_copy['timestamp'] = pd.to_datetime(tweet_data_copy['timestamp'], utc = True)  
tweet_data_copy['timestamp'] = tweet_data_copy['timestamp'] - timedelta(hours=lag)

In [22]:
tweet_data_copy.head()

,Unnamed: 0,id,user,fullname,url,timestamp,replies,likes,retweets,text,polarity,subjectivity
0,0,1132977055300300800,KamdemAbdiel,Abdiel kamdem,NaN,2019-05-27 10:49:14+00:00,0.0,0.0,0.0,È appena uscito un nuovo video! LES CRYPTOMONN...,0.0,0.0
1,1,1132977073402736640,bitcointe,Bitcointe,NaN,2019-05-27 10:49:18+00:00,0.0,0.0,0.0,Cardano: Digitize Currencies; EOS https://t.co...,0.0,0.0
2,2,1132977023893139456,3eyedbran,Bran - 3 Eyed Raven,NaN,2019-05-27 10:49:06+00:00,0.0,2.0,1.0,Another Test tweet that wasn't caught in the s...,0.0,0.0
3,3,1132977089089556481,DetroitCrypto,J. Scardina,NaN,2019-05-27 10:49:22+00:00,0.0,0.0,0.0,Current Crypto Prices! \n\nBTC: $8721.99 USD\n...,0.0,0.4
4,4,1132977092340191232,mmursaleen72,Muhammad Mursaleen,NaN,2019-05-27 10:49:23+00:00,0.0,0.0,0.0,Spiv (Nosar Baz): BITCOIN Is An Asset &amp; NO...,0.0,0.0


In [23]:
#Round timestamps to nearest minute
tweet_data_copy['timestamp'] = tweet_data2['timestamp'].apply(lambda dt: dt.round('min'))

In [25]:
#convert btc data to UTC
btc_data['timestamp'] = pd.to_datetime(btc_data['timestamp'], utc = True)  

In [27]:
#concatinate btc prices and tweets on timestamp
final_data = btc_data.merge(tweet_data2, on="timestamp")

In [28]:
final_data.head()

,Unnamed: 0_x,Close,Volume_(BTC),Volume_(Currency),Weighted_Price,timestamp,Unnamed: 0_y,id,user,fullname,url,replies,likes,retweets,text,polarity,subjectivity
0,3500000,6936.26,1.001363,6944.466619,6935.011702,2018-08-31 11:04:00+00:00,556350,1035528576672530433,bitcoin_whales,Whales NEWS,/bitcoin_whales/status/1035528576672530433,2.0,6.0,0.0,#BTC #CME Futures expiring today 8:00 AM PST. ...,0.000000,0.300000
1,3500001,6944.90,10.379585,72022.107341,6938.823172,2018-08-31 11:05:00+00:00,2628038,1035528763457433600,DolarTrue_,DolarTrue,/DolarTrue_/status/1035528763457433600,0.0,1.0,1.0,#DolarTrue BTC\n31/08/2018 10:04 AM\nBTC Venta...,0.000000,0.000000
2,3500001,6944.90,10.379585,72022.107341,6938.823172,2018-08-31 11:05:00+00:00,2628039,1035528970626682880,KoinKnight,KoinKnight,/KoinKnight/status/1035528970626682880,0.0,0.0,0.0,Price Alert: #NGC is going down.#NGC price has...,-0.400000,0.700000
3,3500007,6939.11,6.495773,45080.950674,6940.044005,2018-08-31 11:11:00+00:00,2628035,1035530457545408512,tr_ticker,Tr Ticker Bot,/tr_ticker/status/1035530457545408512,0.0,0.0,0.0,#Doviz\n-------------------\n#USD : 6.6067\n#E...,0.000000,0.000000
4,3500007,6939.11,6.495773,45080.950674,6940.044005,2018-08-31 11:11:00+00:00,2628036,1035530316989882368,DTradingAcademy,Day Trading Academy,/DTradingAcademy/status/1035530316989882368,0.0,1.0,0.0,"Cryptocurrency Bitcoin -0.5% at $6,945.00 in F...",0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15149445,4146842,7180.83,0.150935,1083.932338,7181.465478,2019-11-23 14:46:00+00:00,20165005,1.19827e+18,theautomatski,Automatski,NaN,0.0,0.0,0.0,The Worlds First Millennium Firm https://t.co/...,0.025000,0.366667
15149446,4146842,7180.83,0.150935,1083.932338,7181.465478,2019-11-23 14:46:00+00:00,20165007,1.19827e+18,digital_mine_,digital mine ⚡🤖👨‍💻🇭🇰🇭🇰🇭🇰,NaN,0.0,0.0,0.0,Daily profit for HODLING BTC since 2013 Data t...,0.000000,0.000000
15149447,4146842,7180.83,0.150935,1083.932338,7181.465478,2019-11-23 14:46:00+00:00,20165009,1.19827e+18,Vizique,Vizique,NaN,0.0,0.0,0.0,Bitcoin Suisse Certificates :) https://t.co/nd...,0.500000,1.000000
15149448,4146842,7180.83,0.150935,1083.932338,7181.465478,2019-11-23 14:46:00+00:00,20165010,1.19827e+18,torusJKL,Gal Buki ($torusJKL),NaN,0.0,0.0,0.0,Register now for the early access of the Codug...,0.100000,0.300000


In [30]:
#drop rows where close is null
final_data = final_data.dropna(axis=0, subset=['Close'])

In [31]:
final_data.head()

,Unnamed: 0_x,Close,Volume_(BTC),Volume_(Currency),Weighted_Price,timestamp,Unnamed: 0_y,id,user,fullname,url,replies,likes,retweets,text,polarity,subjectivity
0,3500000,6936.26,1.001363,6944.466619,6935.011702,2018-08-31 11:04:00+00:00,556350,1035528576672530433,bitcoin_whales,Whales NEWS,/bitcoin_whales/status/1035528576672530433,2.0,6.0,0.0,#BTC #CME Futures expiring today 8:00 AM PST. ...,0.000000,0.300000
1,3500001,6944.90,10.379585,72022.107341,6938.823172,2018-08-31 11:05:00+00:00,2628038,1035528763457433600,DolarTrue_,DolarTrue,/DolarTrue_/status/1035528763457433600,0.0,1.0,1.0,#DolarTrue BTC\n31/08/2018 10:04 AM\nBTC Venta...,0.000000,0.000000
2,3500001,6944.90,10.379585,72022.107341,6938.823172,2018-08-31 11:05:00+00:00,2628039,1035528970626682880,KoinKnight,KoinKnight,/KoinKnight/status/1035528970626682880,0.0,0.0,0.0,Price Alert: #NGC is going down.#NGC price has...,-0.400000,0.700000
3,3500007,6939.11,6.495773,45080.950674,6940.044005,2018-08-31 11:11:00+00:00,2628035,1035530457545408512,tr_ticker,Tr Ticker Bot,/tr_ticker/status/1035530457545408512,0.0,0.0,0.0,#Doviz\n-------------------\n#USD : 6.6067\n#E...,0.000000,0.000000
4,3500007,6939.11,6.495773,45080.950674,6940.044005,2018-08-31 11:11:00+00:00,2628036,1035530316989882368,DTradingAcademy,Day Trading Academy,/DTradingAcademy/status/1035530316989882368,0.0,1.0,0.0,"Cryptocurrency Bitcoin -0.5% at $6,945.00 in F...",0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15149445,4146842,7180.83,0.150935,1083.932338,7181.465478,2019-11-23 14:46:00+00:00,20165005,1.19827e+18,theautomatski,Automatski,NaN,0.0,0.0,0.0,The Worlds First Millennium Firm https://t.co/...,0.025000,0.366667
15149446,4146842,7180.83,0.150935,1083.932338,7181.465478,2019-11-23 14:46:00+00:00,20165007,1.19827e+18,digital_mine_,digital mine ⚡🤖👨‍💻🇭🇰🇭🇰🇭🇰,NaN,0.0,0.0,0.0,Daily profit for HODLING BTC since 2013 Data t...,0.000000,0.000000
15149447,4146842,7180.83,0.150935,1083.932338,7181.465478,2019-11-23 14:46:00+00:00,20165009,1.19827e+18,Vizique,Vizique,NaN,0.0,0.0,0.0,Bitcoin Suisse Certificates :) https://t.co/nd...,0.500000,1.000000
15149448,4146842,7180.83,0.150935,1083.932338,7181.465478,2019-11-23 14:46:00+00:00,20165010,1.19827e+18,torusJKL,Gal Buki ($torusJKL),NaN,0.0,0.0,0.0,Register now for the early access of the Codug...,0.100000,0.300000


### Save to dataset file

In [32]:
final_data.to_csv('./../datasets/tweets_and_prices/'+analyser+'/final_data_lag_'+granularity+'_'+str(lag)+'.csv')